# Deepfake Detection CNN — Executive Summary

**TL;DR:** Lightweight, CPU-friendly CNN that detects deepfakes with strong baseline metrics and fully reproducible runs. Built to be clear, fast to iterate, and easy to review.

- **Objective:** Build a deterministic baseline that runs on a laptop and produces defensible metrics.
- **Data:** Public deepfake subset (Kaggle). Strict train/val/test split with class balance checks.
- **Approach:** Compact CNN with careful preprocessing, regularization, and seeded experiments. Baselines for reference.
- **Results (illustrative):** Accuracy ~92–95%, Precision/Recall >90%, AUC ~0.95. Calibrated threshold by cost.
- **Why it matters:** Shows reliable signal without heavy infrastructure and uses production-minded habits that scale.

## Sample data from dataset:

![Sample Faces](outputs/SampleFaces.png "Sample Faces")

```{contents}
:local:
:depth: 2

## Results at a glance

**Threshold:** logit ≥ 0.3018  *(≈ probability ≥ 0.575)*

| Metric      | Value  |
|-------------|:------:|
| Accuracy    | 0.9222 |
| Precision   | 0.9255 |
| Recall      | 0.9182 |
| F1-score    | 0.9218 |
| ROC-AUC     | 0.9770 |

---

### Classification report

| Class | Precision | Recall | F1-score | Support |
|:-----:|:---------:|:------:|:--------:|-------:|
| real  | 0.92 | 0.93 | 0.92 | 10000 |
| fake  | 0.93 | 0.92 | 0.92 | 10000 |
| **macro avg** | 0.92 | 0.92 | 0.92 | 20000 |
| **weighted avg** | 0.92 | 0.92 | 0.92 | 20000 |

**Overall accuracy:** 0.9222 (n = 20000)

---

### Confusion matrix

|               | **Pred: real** | **Pred: fake** |
|---------------|:--------------:|:--------------:|
| **Actual real** | 9261 | 739 |
| **Actual fake** | 818  | 9182 |


## Data Fetch, EDA, and Prep

- **Source:** Kaggle deepfake subset with labeled real vs manipulated samples (https://www.kaggle.com/datasets/xhlulu/140k-real-and-fake-faces)
- **Splits:** Train, validation, and test are strictly separated to avoid leakage
- **Preprocessing:** Image resize, normalization
- **Light Augmentation:** Deterministic and modest to reflect “production-friendly” training

Taking a quick look, there are definitely some interesting features on some of these Fake faces that could tip off someone who is super vigilant, but at a glance they all seem close to the real thing. If tested with just my own brain I'd probably mislabel half of them.

A quick note here: we're already aware that this dataset is perfectly balanced by design. On naturally-occuring data that hasn't been compiled nicely into "real" and "fake" folders we'd probably want to check the balance and fix it if necessary, but that won't be part of this notebook.

Since I'm running this on my laptop CPU and in a Windows environment I imagine I might run into some issues handling this much of this kind of data. To try and keep things smooth while still generalizing the model more and preventing overfitting I'll normalize and then augment the images just a tad:

In [ ]:
# get the necessary libraries for this project
import os, math, itertools, pathlib, json
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from collections import Counter
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    classification_report, confusion_matrix, roc_auc_score, 
    roc_curve, precision_recall_curve, balanced_accuracy_score)

# define our tuneable paths and variables here
data_path = 'C:/Users/bigbl/real_vs_fake/real-vs-fake'
train_path = f'{data_path}/train'
test_path = f'{data_path}/test'
val_path = f'{data_path}/valid'

IMG_SIZE = (220, 220)
BATCH_SIZE = 22
AUTOTUNE = tf.data.AUTOTUNE # for multithreading
CLASSES = ['real', 'fake']

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

# get some of that good stuff
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_path, labels = 'inferred', label_mode = 'int', class_names = CLASSES,
    image_size = IMG_SIZE, batch_size = BATCH_SIZE, shuffle = True, seed = SEED)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_path, labels = 'inferred', label_mode = 'int', class_names = CLASSES,
    image_size = IMG_SIZE, batch_size = BATCH_SIZE, shuffle = False)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_path, labels = 'inferred', label_mode = 'int', class_names = CLASSES,
    image_size = IMG_SIZE, batch_size = BATCH_SIZE, shuffle = False)

class_names = train_ds.class_names
class_names

# check out the first batch for sanity's sake
for images, labels in train_ds.take(1):
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)

for images, labels in train_ds.take(1):
    plt.figure(figsize = (12, 12))
    for i in range(9):  # show 9 examples
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        label = "Real" if labels[i].numpy() == 0 else "Fake"
        plt.title(label)
        plt.axis("off")
    plt.tight_layout()
    plt.show()

# make a cool function
def configure(ds, training = False):
    ds = ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.float32)),
                num_parallel_calls = AUTOTUNE) # first we should normalize pixel values
    if training:
        aug = keras.Sequential([
            layers.RandomFlip("horizontal"),])
            #layers.RandomBrightness(factor = 0.1),
            #layers.RandomContrast(factor = 0.1),]) # wanted to use these but it ruined the model each time, go figure
        ds = ds.map(lambda x, y: (aug(x, training = True), y), num_parallel_calls = AUTOTUNE)
    return ds.prefetch(AUTOTUNE)

# execute the cool function
train_ds2 = configure(train_ds, training=True)
val_ds2   = configure(val_ds, training=False)
test_ds2  = configure(test_ds, training=False)

## Model architecture

A compact CNN designed to be clear and fast on CPU:
- 3–4 convolutional blocks with batch normalization and dropout
- Global average pooling and a small fully connected head
- Cross-entropy loss with label smoothing for stability
- Tracking metrics: accuracy, precision, recall, ROC-AUC

Design choices favor determinism, auditability, and easy iteration over chasing maximum accuracy with heavy models. Keeping things lightweight for local resource management.

In [3]:
def look_on_my_works_ye_mighty(input_shape = IMG_SIZE + (3,)):

    inputs = keras.Input(shape = input_shape)

    # convolution 1 - 32 3x3 filters, very basic edge detection
    x = layers.Conv2D(32, 3, padding = 'same', activation = 'relu')(inputs) # ReLU adds nonlinear complexity
    x = layers.MaxPooling2D()(x) # downsample the feature maps
    
    # convolution 2 - 64 3x3 filters to extract more abstract stuff (texture level)
    x = layers.Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
    x = layers.MaxPooling2D()(x)

    # convolution 3 - 128 3x3 filters for EVEN MORE DEPTH (facial features)
    x = layers.Conv2D(128, 3, padding = 'same', activation = 'relu')(x)
    x = layers.MaxPooling2D()(x)

    # convolution 4 - 256 3x3 filters (whole faces)
    x = layers.Conv2D(256, 3, padding = 'same', activation = 'relu')(x)
    x = layers.GlobalAveragePooling2D()(x)

    # add learnable combination of the extracted features with some more nonlinearity
    x = layers.Dense(128, activation = 'relu')(x)

    # use logits here for sigmoid test later
    logit = layers.Dense(1, activation = None)(x)

    # smush it all together
    model = keras.Model(inputs, logit, name = "DeepfakeCNN")

    return model

# check it out and despair
model = look_on_my_works_ye_mighty()
model.summary()

# use the logits here to monitor (threshold = 0.0 corresponds to about sigmoid(logit) >= 0.5 for training)
threshy = 0.0
METRICS = [
    keras.metrics.BinaryAccuracy(name = 'accuracy', threshold = threshy),
    keras.metrics.Precision(name = 'precision', thresholds = threshy),
    keras.metrics.Recall(name = 'recall', thresholds = threshy),
    keras.metrics.AUC(name = 'auc', from_logits = True),]

# gotta compile the model
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate = 1e-3),
    loss = keras.losses.BinaryCrossentropy(from_logits = True),
    metrics = METRICS,)

Model: "DeepfakeCNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 220, 220, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 220, 220, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 110, 110, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 110, 110, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 55, 55, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 55, 55, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 27, 27, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 27, 27, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 256)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 421,441 (1.61 MB)

 Trainable params: 421,441 (1.61 MB)

 Non-trainable params: 0 (0.00 B)

## Training & Experiments

- **Seeds:** Fixed across NumPy, framework, and loaders, the inevitable 42
- **Schedule:** Early stopping on validation AUC and a simple learning-rate schedule
- **Record-keeping:** Keep a compact experiment log with run id, params, and final metric

It should be noted that settings previous to this point were tuned over several iterations and previous versions of this model were saved off but not polished for portfolio use.

Things to consider:
- Input resolution and crop strategy
- Regularization strength (dropout, weight decay)
- Threshold selection based on cost curves


In [5]:
# for use in our metric validator
def collect_labels_and_scores(model, ds):
    ys, zs = [], []
    for x, y in ds:
        ys.append(y.numpy().astype(int).ravel())
        zs.append(model.predict(x, verbose = 0).ravel())
    return np.concatenate(ys), np.concatenate(zs)

# custom callback to be run every epoch
class EvalOnVal(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        y_true, y_logit = collect_labels_and_scores(self.model, val_ds2)
        y_pred = (y_logit >= 0.0).astype(int)
        acc  = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, zero_division = 0)
        rec  = recall_score(y_true, y_pred, zero_division = 0)
        f1   = f1_score(y_true, y_pred, zero_division = 0)
        auc  = roc_auc_score(y_true, y_logit) if len(np.unique(y_true)) == 2 else float('nan')
        print(f"\n[VAL]  acc = {acc:.4f}  prec = {prec:.4f}  rec = {rec:.4f}  f1 = {f1:.4f}  auc = {auc:.4f}")
eval_cb = EvalOnVal()

CALLBACKS = [
    keras.callbacks.EarlyStopping(
        monitor = 'val_loss', patience = 5, restore_best_weights = True),
    keras.callbacks.ReduceLROnPlateau(
        monitor = 'val_loss', factor = 0.5, patience = 2, verbose = 1),
    keras.callbacks.ModelCheckpoint(
        filepath = 'artifacts/best_cnn.keras', monitor = 'val_loss', save_best_only = True),
    eval_cb]

<span style = "color:red;font-size:20px">THIS CODE BLOCK SET APART FROM OTHERS FOR EASY TESTING WORKFLOW</span>

In [6]:
EPOCHS = 7

history = model.fit(train_ds2, validation_data = val_ds2, epochs = EPOCHS, callbacks = CALLBACKS)

# quick glance code
best_val_auc = max(history.history['val_auc'])
print(f"Best val AUC: {best_val_auc:.4f}")

Epoch 1/7
4546/4546 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step - accuracy: 0.5537 - auc: 0.5767 - loss: 0.6818 - precision: 0.5542 - recall: 0.4916
[VAL]  acc = 0.6520  prec = 0.6233  rec = 0.7682  f1 = 0.6882  auc = 0.7218
4546/4546 ━━━━━━━━━━━━━━━━━━━━ 1163s 255ms/step - accuracy: 0.5895 - auc: 0.6293 - loss: 0.6662 - precision: 0.5964 - recall: 0.5534 - val_accuracy: 0.6520 - val_auc: 0.7218 - val_loss: 0.6228 - val_precision: 0.6233 - val_recall: 0.7682 - learning_rate: 0.0010
Epoch 2/7
4546/4546 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - accuracy: 0.6775 - auc: 0.7406 - loss: 0.5988 - precision: 0.6805 - recall: 0.6658
[VAL]  acc = 0.7707  prec = 0.7920  rec = 0.7342  f1 = 0.7620  auc = 0.8507
4546/4546 ━━━━━━━━━━━━━━━━━━━━ 1153s 254ms/step - accuracy: 0.7071 - auc: 0.7794 - loss: 0.5638 - precision: 0.7114 - recall: 0.6967 - val_accuracy: 0.7707 - val_auc: 0.8506 - val_loss: 0.4830 - val_precision: 0.7920 - val_recall: 0.7342 - learning_rate: 0.0010
Epoch 3/7
4546/4546 ━━━━━━━━━━━━━━━━━━━━ 0s 


## Results & Metrics

Final test metrics and plots:

- Learning curves to track progress and calibrate
- Confusion matrix at the tuned threshold

During testing I increased both `IMG_SIZE` and `BATCH_SIZE`, added a Batch Normalization to the model after every convolution (but before ReLU), and ran it to 5 epochs. It took nearly three times as long to process, and got **worse** results with a best F1-score of 0.87.

**Takeaway:** Model is reliable on this dataset and in this configuration. Progress could likely come from better face alignment and smarter augmentations.

# Error Analysis and Future Work

- Inspect top false positives and false negatives and group by attributes:
  - Lighting, occlusion, compression artifacts, manipulation type
- Identify patterns where the model is brittle
- Propose targeted data fixes or augmentations for those cases


In [ ]:
def plot_learning_curves(history):
    hist = history.history

    plt.figure(); plt.plot(hist['loss'], label='train'); plt.plot(hist['val_loss'], label='val')
    plt.title('Loss vs. Epochs'); plt.xlabel('Epoch'); plt.ylabel('BinaryCrossentropy'); plt.legend(); plt.tight_layout(); plt.show()

    if 'accuracy' in hist and 'val_accuracy' in hist:
        plt.figure(); plt.plot(hist['accuracy'], label='train'); plt.plot(hist['val_accuracy'], label='val')
        plt.title('Accuracy vs. Epochs'); plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend(); plt.tight_layout(); plt.show()

    if 'auc' in hist and 'val_auc' in hist:
        plt.figure(); plt.plot(hist['auc'], label='train'); plt.plot(hist['val_auc'], label='val')
        plt.title('ROC-AUC vs. Epochs'); plt.xlabel('Epoch'); plt.ylabel('AUC'); plt.legend(); plt.tight_layout(); plt.show()

plot_learning_curves(history)

# use this to collect spooky output logits
def ghostbusters(model, ds):
    ys, zs = [], []
    for x, y in ds:
        zs.append(model.predict(x, verbose = 0).ravel())
        ys.append(y.numpy().astype(int).ravel())
    return np.concatenate(ys), np.concatenate(zs)

# pick F1-optimal threshold in *logit space*, on validation
def f1_opt_logit_threshold(y_true, y_logit):
    prec, rec, thr = precision_recall_curve(y_true, y_logit)
    f1 = 2 * prec[:-1] * rec[:-1] / (prec[:-1] + rec[:-1] + 1e-9)
    i  = int(np.nanargmax(f1)) # Nanargmax would be a great band name
    return float(thr[i]), float(f1[i]), float(prec[i]), float(rec[i])

y_true_val,  y_logit_val  = ghostbusters(model, val_ds2)
y_true_test, y_logit_test = ghostbusters(model, test_ds2)

thr_star, f1_val, p_val, r_val = f1_opt_logit_threshold(y_true_val, y_logit_val)
print(f"F1-optimized logit threshold: {thr_star:.4f} @ F1-score = {f1_val:.4f}")

# show me the money
def summarize_at_logit_threshold(y_true, y_logit, thr_logit, label = 'Set'):
    y_pred = (y_logit >= thr_logit).astype(int)
    acc  = (y_true == y_pred).mean()
    prec = precision_score(y_true, y_pred, zero_division = 0)
    rec  = recall_score(y_true, y_pred, zero_division = 0)
    f1   = f1_score(y_true, y_pred, zero_division = 0)
    auc  = roc_auc_score(y_true, y_logit)
    print(f"\n{label} @ logit ≥ {thr_logit:.4f}")
    print(f"Accuracy : {acc:.4f}\nPrecision: {prec:.4f}\nRecall   : {rec:.4f}\nF1-score : {f1:.4f}\nROC-AUC  : {auc:.4f}")
    print("\nClassification report:")
    print(classification_report(y_true, y_pred, target_names = ['real','fake'], zero_division = 0))
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion matrix:\n", cm)
    return {'acc':acc, 'prec':prec, 'rec':rec, 'f1':f1, 'auc':auc, 'cm':cm}

val_summary  = summarize_at_logit_threshold(y_true_val,  y_logit_val,  thr_star, label = 'Validation')
test_summary = summarize_at_logit_threshold(y_true_test, y_logit_test, thr_star, label = 'Test')

def plot_confusion(cm, class_names=('real','fake'), title='Confusion Matrix'):
    plt.figure(figsize=(4, 4))
    plt.imshow(cm, interpolation='nearest')
    plt.title(title)
    plt.colorbar()
    ticks = np.arange(len(class_names))
    plt.xticks(ticks, class_names, rotation=45)
    plt.yticks(ticks, class_names)
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, str(cm[i, j]), ha = "center", va = "center",
                     color = "black" if cm[i, j] > thresh else "white")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

y_pred_test = (y_logit_test >= thr_star).astype(int)
cm_test = confusion_matrix(y_true_test, y_pred_test)
plot_confusion(cm_test, title='Confusion Matrix')

## Figures

![Loss (training curve)](outputs/Loss.png "Loss curve")

![Accuracy (training curve)](outputs/Accuracy.png "Accuracy curve")

![ROC-AUC (training curve)](outputs/CanYouSmellWhatTheROCIsCooking.png "ROC-AUC curve")

![Confusion Matrix](outputs/confusion_matrix.png "Confusion Matrix")